METADATA
<br>
000CT: Paris 6ème - Saint Germain des Prés
<br>
Source: www.infoclimat.fr
<br>
License Ouverte/Open License: http://wiki.data.gouv.fr/images/9/9d/Licence_Ouverte.pdf
<br>
<br>
temperature = degC
<br>
pression = hPa
<br>
humidite = %
<br>
point_de_rosee = degC
<br>
vent_moyen = km/h
<br>
vent_rafales = km/h
<br>
vent_direction = deg
<br>
pluie_3h = mm
<br>
pluie_1h = mm

Importation des librairies

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, roc_curve, auc, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import os

Lecture du fichier `export_infoclimat.csv`

In [ ]:
data = pd.read_csv('export_infoclimat.csv', sep=';')
data

Listing des valeurs `null` et `NaN`

In [ ]:
missing_values = data.isnull().sum()
missing_values

Attribution des données médian à la place de `NaN` pour les colonnes `tempeareture`, `humidite`, `point_de_rosee`, `vent_moyen` et `vent_direction`
<br>
Retrait de la colonne `vent_rafales`
<br>
Attribution de la valeur `0` dans les colonnes `pluie_3h` et `pluie_1h` si `NaN`

In [ ]:
# Fonction pour remplacer les valeurs manquantes par la médiane des 3 précédents et 3 suivants
def remplacer_par_median_locale(df, colonne):
    for i in df[colonne].index[df[colonne].isnull()]:
        valeurs_valides = df[colonne][max(i-3,0) : min(i+4, len(df[colonne]))]  # Récupérer les valeurs autour de la valeur manquante
        median_locale = valeurs_valides.median()  # Calculer la médiane locale
        df.at[i, colonne] = median_locale  # Remplacer la valeur manquante par la médiane locale
        pass
    return df

# Appliquer la fonction aux colonnes spécifiées
colonnes = ['temperature', 'humidite', 'point_de_rosee', 'vent_moyen', 'vent_direction']
for col in colonnes:
    data = remplacer_par_median_locale(data, col)
    pass

data.drop(columns=['vent_rafales'], inplace=True)

data['pluie_3h'] = data['pluie_3h'].fillna(0)
data['pluie_1h'] = data['pluie_1h'].fillna(0)

data.isnull().sum()

Retrait de la colonne `station_id` et mise en format `datetime` de la colonne `dh_utc`

In [ ]:
data.drop(columns=['station_id'], inplace=True)
data['dh_utc'] = pd.to_datetime(data['dh_utc'])
data

Calcul de l'indice de chaleur

In [ ]:
# data['dh_utc'] = pd.to_datetime(data['dh_utc'])

# # Ajouter des caractéristiques pour l'heure et le mois
# data['heure'] = data['dh_utc'].dt.hour
# data['mois'] = data['dh_utc'].dt.month

# # Fonction pour calculer l'indice de chaleur (approximation simple)
# def calculer_indice_chaleur(temp_celsius, humidite):
#     temp_fahrenheit = 9/5 * temp_celsius + 32
#     indice = -42.379 + 2.04901523*temp_fahrenheit + 10.14333127*humidite \
#             - 0.22475541*temp_fahrenheit*humidite - 6.83783*10**-3*temp_fahrenheit**2 \
#             - 5.481717*10**-2*humidite**2 + 1.22874*10**-3*temp_fahrenheit**2*humidite \
#             + 8.5282*10**-4*temp_fahrenheit*humidite**2 - 1.99*10**-6*temp_fahrenheit**2*humidite**2
#     return (indice - 32) * 5/9  # Convertir en Celsius pour le retour

# # Ajout de l'indice de chaleur au DataFrame
# data['indice_chaleur'] = data.apply(lambda row: calculer_indice_chaleur(row['temperature'], row['humidite']), axis=1)

# # Vérification des premières lignes pour s'assurer que la colonne a été ajoutée correctement
# data[['temperature', 'humidite', 'indice_chaleur']].head()

In [ ]:
# data

### Classification de la colonne `pluie_1h` selon les paramètres `temperature`, `pression`, `humidite` et `vent_moyen`

In [ ]:
# Récupération des paramètres
X = data[['temperature', 'pression', 'humidite', 'vent_moyen']]
y = (data['pluie_1h'] > 0).astype(int) # 1 = pluie 0 = pas pluie

# Séparation des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Création et entraînement du modèle de forêt aléatoire
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Prédiction sur l'ensemble de test
y_pred = model.predict(X_test)

# Évaluation de la performance
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print(classification_report(y_test, y_pred))

In [ ]:
# Création du répertoire models s'il n'existe pas
if not os.path.exists('./models'):
    os.mkdir('models')
    pass

# Stockage du model de classification en pickle
pickle.dump(model, open('./models/classification_model.pickle', 'wb'))

In [ ]:
# Calcul de la matrice de confusion
cm = confusion_matrix(y_test, y_pred)

# Affichage de la matrice de confusion avec Seaborn
plt.figure(figsize=(10,7))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Non Pluvieux', 'Pluvieux'], yticklabels=['Non Pluvieux', 'Pluvieux'])
plt.xlabel('Prédit')
plt.ylabel('Réel')
plt.title('Matrice de Confusion')
plt.show()

In [ ]:
# Calcul des taux de faux positifs et vrais positifs
fpr, tpr, thresholds = roc_curve(y_test, model.predict_proba(X_test)[:,1])

# Calcul de l'aire sous la courbe ROC
roc_auc = auc(fpr, tpr)

# Affichage de la courbe ROC
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Taux de Faux Positif')
plt.ylabel('Taux de Vrai Positif')
plt.title('Courbe ROC')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# Affichage de l'importance des caractéristiques
feature_importances = pd.Series(model.feature_importances_, index=X_train.columns).sort_values(ascending=False)

plt.figure(figsize=(12,8))
feature_importances.plot(kind='bar')
plt.title('Importance des Caractéristiques')
plt.xlabel('Caractéristiques')
plt.ylabel('Importance')
plt.show()

### Réutilisation du model

In [ ]:
# Chargement du modèle depuis le fichier pickle
model = pickle.load(open('./models/classification_model.pickle', 'rb'))

In [ ]:
# Initialisation des données de test
temperature = 15
pression = 1010
humidite = 80
vent_moyen = 5

exemple_entree = pd.DataFrame([[temperature, pression, humidite, vent_moyen]], columns=['temperature', 'pression', 'humidite', 'vent_moyen'])
exemple_entree

In [ ]:
# Prédiction selon mes données de test
prediction = model.predict(exemple_entree)

# Afficher le résultat de la prédiction
if prediction[0] == 1:
    print("Il va probablement pleuvoir.")
else:
    print("Il ne devrait pas pleuvoir.")